In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate, zip_longest, pairwise, tee
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set
import functools
np.set_printoptions(linewidth=160)

In [3]:
DAY = 15

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'Sensor at x=1555825, y=18926: closest beacon is at x=1498426, y=-85030\nSensor at'
Split Data:
['Sensor at x=1555825, y=18926: closest beacon is at x=1498426, y=-85030',
 'Sensor at x=697941, y=3552290: closest beacon is at x=595451, y=3788543',
 'Sensor at x=3997971, y=2461001: closest beacon is at x=3951198, y=2418718',
 'Sensor at x=3818312, y=282332: closest beacon is at x=4823751, y=1061753',
 'Sensor at x=2874142, y=3053631: closest beacon is at x=3074353, y=3516891',
 'Sensor at x=1704479, y=2132468: closest beacon is at x=1749091, y=2000000',
 'Sensor at x=3904910, y=2080560: closest beacon is at x=3951198, y=2418718',
 'Sensor at x=657061, y=3898803: closest beacon is at x=595451, y=3788543',
 'Sensor at x=3084398, y=3751092: closest beacon is at x=3074353, y=3516891',
 'Sensor at x=2582061, y=972407: closest beacon is at x=1749091, y=2000000']


In [5]:
test_data = [
"Sensor at x=8, y=7: closest beacon is at x=2, y=10",
"Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
"Sensor at x=9, y=16: closest beacon is at x=10, y=16",
"Sensor at x=13, y=2: closest beacon is at x=15, y=3",
"Sensor at x=12, y=14: closest beacon is at x=10, y=16",
"Sensor at x=10, y=20: closest beacon is at x=10, y=16",
"Sensor at x=14, y=17: closest beacon is at x=10, y=16",
"Sensor at x=2, y=0: closest beacon is at x=2, y=10",
"Sensor at x=0, y=11: closest beacon is at x=2, y=10",
"Sensor at x=20, y=14: closest beacon is at x=25, y=17",
"Sensor at x=17, y=20: closest beacon is at x=21, y=22",
"Sensor at x=16, y=7: closest beacon is at x=15, y=3",
"Sensor at x=14, y=3: closest beacon is at x=15, y=3",
"Sensor at x=20, y=1: closest beacon is at x=15, y=3",
]

In [20]:
class Square(IntEnum):
    EMPTY = 0
    SENSOR = 1
    BEACON = 2
    CONFIRMED = 3

In [21]:
line_re = re.compile(r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)")

In [22]:
char_dict = {Square.EMPTY:".", Square.BEACON:"B", Square.SENSOR:"S", Square.CONFIRMED:"#"}

In [132]:
def parse(str_data):
    ret = []
    for l in str_data:
        ret.append(list(map(int, line_re.match(l).groups())))
    return ret

In [133]:
def create_grid(int_data):
    minx = np.inf
    miny = np.inf
    maxx = -np.inf
    maxy = -np.inf
    min_man = -np.inf
    
    for (sx,sy,bx,by) in int_data:
        minx = min(minx, sx)
        minx = min(minx, bx)
        
        maxx = max(maxx, sx)
        maxx = max(maxx, bx)
        
        miny = min(miny, sy)
        miny = min(miny, by)
        
        maxy = max(maxy, sy)
        maxy = max(maxy, by)
        
        min_min = min((abs(by-sy) + abs(bx-sx)), min_man)
    
    grid = np.full((maxy-miny+1, maxx-minx+1), fill_value=Square.EMPTY)
    
    y_offset = miny
    x_offset = minx
    
    for (sx,sy,bx,by) in int_data:
        grid[(sy-y_offset),(sx-x_offset)] = Square.SENSOR
        grid[(by-y_offset),(bx-x_offset)] = Square.BEACON

        
    y_shape, x_shape = grid.shape
    
    approx_max_man_dist = min_min
    
    #expand vertically
    vertical_padding = np.full((approx_max_man_dist, x_shape), fill_value=Square.EMPTY)
    
    grid = np.vstack((vertical_padding, grid))
    grid = np.vstack((grid, vertical_padding))
    y_offset -= approx_max_man_dist
    
    y_shape, x_shape = grid.shape
    
    horizontal_padding = np.full((y_shape, approx_max_man_dist), fill_value=Square.EMPTY)
    
    grid = np.hstack((horizontal_padding, grid))
    grid = np.hstack((grid, horizontal_padding))
    x_offset -= approx_max_man_dist
        
    return grid, (y_offset, x_offset), int_data    

In [134]:
def print_grid(grid, offsets):
    y_off, x_off = offsets
    indent = 8
    y,x = grid.shape
    total_string = "\n".join([" "*indent + ''.join(row) for row in np.array([list(f"{n:>5}") for n in range(x_off, x_off + x)]).T]) + "\n"
    for i, row in enumerate(grid,start=y_off):
        row_string = "".join([char_dict[x] for x in row])
        row_string = f"{i:>7} {row_string}"
        total_string = total_string + row_string +"\n"
    print(total_string)

In [135]:
#https://stackoverflow.com/a/58348675/2597564
def diamond(n):
    a = np.arange(n)
    b = np.minimum(a,a[::-1])
    return (b[:,None]+b)>=(n-1)//2

In [136]:
def confirm(grid, offsets, int_data):
    y_off, x_off = offsets

    for i, (o_sx,o_sy,o_bx,o_by) in enumerate(int_data):
        grid_y, grid_x = grid.shape
        man_dist = abs(o_by-o_sy) + abs(o_bx-o_sx)
            
        diamond_mask = diamond((man_dist * 2)+1)
        diamond_mask_i = np.indices(diamond_mask.shape)

        diamond_mask = diamond_mask.flatten()
        
        diamond_mask_i[0] += (o_sy - y_off - man_dist)
        diamond_mask_i[1] += (o_sx - x_off - man_dist)
        
        y_i = diamond_mask_i[0].flatten()
        x_i = diamond_mask_i[1].flatten()
        
        y_i = y_i[np.nonzero(diamond_mask)]
        x_i = x_i[np.nonzero(diamond_mask)]
        
        overlap = grid[y_i, x_i]
        y_i = y_i[np.where(overlap == Square.EMPTY)]
        x_i = x_i[np.where(overlap == Square.EMPTY)]
        
        grid[y_i,x_i] = Square.CONFIRMED

        # print_grid(grid, (y_off, x_off))
        
    #trim grid
    return grid, (y_off, x_off)

In [137]:
grid, (y_off, x_off) = confirm(*create_grid(parse(data)))

MemoryError: Unable to allocate 86.7 TiB for an array with shape (4082599, 5834177) and data type int32

In [114]:
guess_row = 2000000

In [130]:
len(np.nonzero(grid[guess_row-y_off] == Square.CONFIRMED)[0])

26

In [107]:
def trim_grid(grid, offsets):
    y_off, x_off = offsets
    print(grid.shape)

    y_i = np.any(grid, axis=1)
    n_y_o = np.argmax(y_i>0)
    y_off += n_y_o
    y = np.nonzero(y_i)

    partial_grid = grid[y].T

    x_i = np.any(partial_grid, axis=1)
    n_x_o = np.argmax(x_i>0)
    x_off += n_x_o
    x = np.nonzero(x_i)

    trim_grid = partial_grid[x].T
    print(trim_grid.shape)
    print_grid(trim_grid, (y_off, x_off))

In [6]:
def parse(str_data):
    return [[tuple(map(int, n.strip().split(","))) for n in l.split("->")] for l in str_data]

In [7]:
SAND = 1
DROP = 2
EMPTY = 0
ROCK = np.inf

In [8]:
char_dict = {EMPTY:".", ROCK:"#", SAND:"O", DROP:"+"}

In [9]:
def print_cave(cave):
    print("\n".join(["".join([char_dict[x] for x in row]) for row in cave]))

In [10]:
def make_rocks(int_data, ground=False):
    minx = np.inf
    maxx = maxy = 0
    for l in int_data:
        for a,b in l:
            minx = min(minx, a)
            maxx = max(maxx, a)
            maxy = max(maxy, b)

    cave = np.zeros((maxy + 1, (maxx + 1) - minx ))
     
    for l in int_data:
        for (x1, y1), (x2, y2) in pairwise(l):
            x1 -= minx
            x2 -= minx          
            if x1 == x2:
                y1, y2 = sorted([y1,y2])
                cave[y1:(y2+1), x1] = ROCK
            elif y1 == y2:
                x1, x2 = sorted([x1,x2])
                cave[y1, x1:(x2+1)] = ROCK
            else:
                raise ValueError
    
    cave = np.vstack((np.zeros((1, cave.shape[1])), cave))
    maxy += 1
    
    if ground:
        cave = np.hstack((cave, np.zeros((maxy + 1, maxy))))
        cave = np.hstack((np.zeros((maxy + 1, maxy)), cave))
        minx -= maxy
        maxx += maxy
    
        cave = np.vstack((cave, np.zeros((1, cave.shape[1]))))
        cave = np.vstack((cave, np.full((1, cave.shape[1]), ROCK)))
        maxy += 2
    
    return (cave, minx)

In [11]:
def place_sand(cave, py, px):
    try:
        neigh = np.array(neighbors(cave, (py, px))).T
        y_low, x_low = tuple(neigh[6:].T)
        # straight below is empty
        if cave[y_low[1], x_low[1]] == EMPTY:
            # print("empty")
            return place_sand(cave, py+1, px)
        # all non-empty, return false with new cave
        elif np.count_nonzero(cave[y_low, x_low] != EMPTY) == 3:
            # print("settle")
            cave[py,px]= SAND
            return cave, False
        # try first non-empty slow below
        else:
            i = np.argmax(cave[y_low, x_low] == 0)
            return place_sand(cave, y_low[i], x_low[i])    
    except IndexError:
        return cave, True

In [13]:
def drop_sand(cave, minx, drop_pt = (500,1)):
    dropx, dropy = drop_pt
    dropx -= minx
    cave[dropy, dropx] = 2
    
    end_found = False
    
    while not end_found:
        py, px = (dropy, dropx)
        cave, end_found = place_sand(cave, py, px)
        if cave[dropy,dropx]==SAND:
            end_found=True
    print_cave(cave)
    return np.count_nonzero(cave == SAND)

In [16]:
drop_sand(*make_rocks(parse(data)))

..................................................................................
............................................................+.....................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
..................................................................................
....

843

In [441]:
submit_answer(DAY, 1, Out[max(Out.keys())])

True

In [17]:
drop_sand(*make_rocks(parse(data), ground=True))

......................................................................................................................................................................................................................................................................................................................................................................................................................................
......................................................................................................................................................................................................................................O...............................................................................................................................................................................................
..........................................................................................................................................................

27625

In [18]:
submit_answer(DAY, 2, Out[max(Out.keys())])

27625
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit.You have completed Day 14! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True